In [ ]:
def create_mask_otsu_test(image):
	"""
	Create an enhanced binary mask using an improved preprocessing pipeline:
	1. Convert to grayscale.
	2. Enhance contrast using CLAHE.
	3. Denoise with a bilateral filter.
	4. Sharpen using an unsharp mask filter.
	5. Optionally smooth with a Gaussian blur.
	6. Apply Otsu's thresholding.
	7. Clean up with morphological operations.
	"""
	# Convert image to grayscale
	gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

	# Enhance local contrast using CLAHE
	clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
	enhanced = clahe.apply(gray)

	# Use a bilateral filter to reduce noise while preserving edges
	denoised = cv2.bilateralFilter(enhanced, d=9, sigmaColor=75, sigmaSpace=75)

	# Sharpen the image using an unsharp masking kernel
	sharpening_kernel = np.array([[-1, -1, -1],
	                              [-1, 9, -1],
	                              [-1, -1, -1]])
	sharpened = cv2.filter2D(denoised, -1, sharpening_kernel)

	# Optional: Apply Gaussian Blur to reduce any high-frequency artifacts
	blurred = cv2.GaussianBlur(sharpened, (5, 5), 0)

	# Apply Otsu's thresholding to create the binary mask
	_, mask = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

	# Use morphological opening to remove small noise artifacts from the mask
	kernel_morph = np.ones((3, 3), np.uint8)
	mask_clean = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_morph, iterations=1)

	return mask_clean


def create_mask_watershed(image):
	"""Create mask using Watershed segmentation"""
	# Convert to grayscale
	gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
	# Apply Gaussian blur
	blurred = cv2.GaussianBlur(gray, (5, 5), 0)
	# Otsu's thresholding for markers
	_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

	# Noise removal using morphological operations
	kernel = np.ones((3, 3), np.uint8)
	opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

	# Sure background area
	sure_bg = cv2.dilate(opening, kernel, iterations=3)

	# Finding sure foreground area
	dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
	_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
	sure_fg = np.uint8(sure_fg)

	# Finding unknown region
	unknown = cv2.subtract(sure_bg, sure_fg)

	# Marker labelling
	_, markers = cv2.connectedComponents(sure_fg)
	markers = markers + 1
	markers[unknown == 255] = 0

	# Apply watershed
	markers = cv2.watershed(image, markers)
	mask = np.zeros_like(gray)
	mask[markers > 1] = 255
	return mask


def create_mask_adaptive(image):
	"""Create mask using adaptive thresholding"""
	gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
	blurred = cv2.GaussianBlur(gray, (5, 5), 0)
	mask = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
	                             cv2.THRESH_BINARY_INV, 11, 2)
	return mask


def process_and_save_results(image_path, output_dir):
	"""Process an image with different segmentation methods and save results"""
	# Read and preprocess image
	image = cv2.imread(str(image_path))
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

	# Create output directory if it doesn't exist
	output_dir = Path(output_dir)
	output_dir.mkdir(parents=True, exist_ok=True)

	# Get base filename
	base_name = image_path.stem

	# Apply different segmentation methods
	masks = {
			'otsu':      create_mask_otsu_test(image),
			'watershed': create_mask_watershed(image),
			'adaptive':  create_mask_adaptive(image)
	}

	# Create figure to display results
	plt.figure(figsize=(15, 5))

	# Plot original image
	plt.subplot(141)
	plt.imshow(image)
	plt.title('Original')
	plt.axis('off')

	# Plot masks
	for i, (method, mask) in enumerate(masks.items(), 2):
		plt.subplot(1, 4, i)
		plt.imshow(mask, cmap='gray')
		plt.title(method.capitalize())
		plt.axis('off')

		# Save individual mask
		cv2.imwrite(str(output_dir / f"{base_name}_{method}_mask.png"), mask)

	# Save comparison figure
	plt.savefig(str(output_dir / f"{base_name}_comparison.png"))
	plt.close()


def maskTest():
	# Define paths
	validation_dir = Path(r"E:\Capstone Skin Cancer Project\Datasets\All Images\Mask Validation")
	output_dir = validation_dir / "segmentation_results"

	# Process each image in the validation directory
	for image_path in validation_dir.glob("*.jpg"):
		try:
			process_and_save_results(image_path, output_dir)
			print(f"Processed {image_path.name}")
		except Exception as e:
			print(f"Error processing {image_path.name}: {str(e)}")
	print("Segmentation results saved.")


maskTest()